# Import Required Libraries

In [23]:
from fuzzywuzzy import fuzz
import pandas as pd
import re # use regular expression (regex) operations on top of fuzzscore
import time

# Use Case 1: Match BBW Products with BBW Orders

In [30]:
# === get product & order data === #
products_df = pd.read_csv('../datasets/products.csv')
products_df = products_df[['product_name','shopify_product_id','product_type']]

products_df["product_name"].value_counts() # we see that there are duplicates
products_df = products_df.drop_duplicates(subset=['product_name'])
print('Products df shape: ' + str(products_df.shape))

orders_df = pd.read_csv('../datasets/cleaned_orders.csv')
print('Orders df shape: ' + str(orders_df.shape))

# === merge product & order data to find exact matches of product name === #
merged = orders_df.merge(products_df, left_on='item_id', right_on="product_name", how="left")
print('Merged df shape: ' + str(merged.shape))

Products df shape: (2355, 3)
Orders df shape: (19467, 13)
Merged df shape: (19467, 16)


In [31]:
# merged_cleaned: contains all the orders that have exact matches to product name from product data
merged_cleaned = merged.dropna(subset=['shopify_product_id'])
print('merged_cleaned df shape: ' + str(merged_cleaned.shape))

# dropped: contains all the orders that don't have exact matches
dropped = merged[merged['shopify_product_id'].isna()]
print('dropped df shape: ' + str(dropped.shape))

# === to check which order items have been dropped === #
# values = dropped['item_id'].value_counts()
# values.to_csv('datasets/items_dropped.csv')

merged_cleaned df shape: (10444, 16)
dropped df shape: (9023, 16)


In [32]:
# products dictionary, key: product_name, values: { shopify_product_id, product_type } 
products_dict = products_df.set_index('product_name').to_dict('index') 

In [33]:
copied_dropped = dropped.copy() # orders with no exact product name matches

for index, row in dropped.iterrows():
    # preliminary checks to reduce comp power -> check if substring inside pdt dict keys
    volume_pattern = r'(- \d+\s*(ml|ML|l|L))'  # Match digits followed by "ml,", "ML", "L," or "l"
    year_pattern = r'(\b\d{4}\b)'  # Match year/vintage
    # Remove volume & year patterns from the string
    string = re.sub(volume_pattern, '', row['item_id'], flags=re.IGNORECASE) # case-insensitive
    string = re.sub(year_pattern, '', string)
    res = [key for key in products_dict.keys() if string in key]

    fuzzthreshold = 70 # threshold for fuzzy score - currently an arbitrary number

    if len(res) == 0: # if the new string gives no matches, do fuzzy score with pdt dict
        max_score = 0
        best_match = ""
        for key in products_dict.keys():
            if fuzz.ratio(row['item_id'], key) > max_score and max_score >= fuzzthreshold:
                best_match = key
                max_score = fuzz.ratio(row['item_id'], key)

    if len(res) >= 1:
        scores = []
        for pdt in res: # find similarity score based on ORIGINAL pdt name from orders data
            scores.append(fuzz.ratio(row['item_id'], pdt))
        max_score = max(scores) # find max score - best match
        best_match = res[scores.index(max_score)]

    if max_score >= fuzzthreshold:
        copied_dropped.at[index, 'product_name'] = best_match
        copied_dropped.at[index, 'shopify_product_id'] = products_dict[best_match]['shopify_product_id']
        copied_dropped.at[index, 'product_type'] = products_dict[best_match]['product_type']

In [34]:
# merged_cleaned2: contains orders that have good-enough product name matches
merged_cleaned2 = copied_dropped.dropna(subset=['shopify_product_id'])
print('merged_cleaned2 df shape: ' + str(merged_cleaned2.shape))

# final: all matched products from 1st & 2nd round of matching
final = pd.concat([merged_cleaned, merged_cleaned2], ignore_index=True)
print('final df shape: ' + str(final.shape))

merged_cleaned2 df shape: (3744, 16)
final df shape: (14188, 16)


In [36]:
final.to_csv('../datasets/matched_orders.csv')

# Use Case 2: Match BBW Products with Competitors' Products